# Практика 8: Методы защиты от атак на модели ИИ

## Выполнил студент группы ББМО-01-23 Егоров Ю.А.

## Загрузка и подготовка данных
В этом блоке происходит загрузка данных MNIST, их нормализация и преобразование меток в one-hot encoding.

In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка и ограничение объёма данных
def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0  # Нормализация
    y_train = tf.keras.utils.to_categorical(y_train, 10)  # One-hot encoding
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    return x_train[:1000], y_train[:1000], x_test, y_test

# Загрузка данных
train_images, train_labels, test_images, test_labels = load_and_preprocess_data()

# Определение моделей
В этом блоке определяются базовая модель, модель с градиентной маскировкой и модель с регуляризацией.

In [10]:
# Базовая модель
def build_model():
    nn_model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    nn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return nn_model

# Модель с градиентной маскировкой
def build_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        tf.keras.layers.Activation('softplus')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Модель с регуляризацией и нормализацией
def build_regularized_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Адаптивное обучение с противоречивыми примерами
def robust_training(neural_net, dataset_images, dataset_labels, eps_value):
    for ep in range(5):  # Небольшое количество эпох для демонстрации
        for start in range(0, len(dataset_images), 32):
            subset_images = dataset_images[start:start+32]
            subset_labels = dataset_labels[start:start+32]
            adversarial_set = create_adversarial_samples(neural_net, subset_images, subset_labels, eps_value)
            combined_imgs = np.concatenate([subset_images, adversarial_set], axis=0)
            combined_lbls = np.concatenate([subset_labels, subset_labels], axis=0)
            neural_net.train_on_batch(combined_imgs, combined_lbls)


# Реализация FGSM атаки
Генерация противоречивых примеров с использованием FGSM.

In [11]:
# FGSM атака
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * tf.sign(gradient)
    adversarial_image = image + perturbation
    return tf.clip_by_value(adversarial_image, 0, 1)

# Генерация противоречивых примеров
def create_adversarial_samples(model, input_images, input_labels, eps):
    generated_samples = []
    for idx in range(len(input_images)):
        img = tf.convert_to_tensor(input_images[idx].reshape((1, 28, 28, 1)), dtype=tf.float32)
        lbl = tf.convert_to_tensor(input_labels[idx].reshape((1, 10)), dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(img)
            preds = model(img)
            cost = tf.keras.losses.categorical_crossentropy(lbl, preds)
        grad = tape.gradient(cost, img)
        adversarial_img = fgsm_attack(img, eps, grad)
        generated_samples.append(adversarial_img.numpy().reshape(28, 28))
    return np.array(generated_samples)

# Обучение и сохранение моделей
Этот блок содержит процесс обучения моделей и их сохранения.

In [12]:
# Обучение базовой модели
base_model = build_model()
base_model.fit(train_images, train_labels, epochs=5)
base_model.save('base_model.h5')

# Обучение модели с градиентной маскировкой
masked_model = build_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
masked_model.save('masked_model.h5')

# Обучение модели с регуляризацией и нормализацией
regularized_model = build_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
regularized_model.save('regularized_model.h5')

# Обучение модели с использованием противоречивых примеров
adv_model = build_model()
robust_training(adv_model, train_images, train_labels, eps_value=0.1)
adv_model.save('adversarial_trained_model.h5') 


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4398 - loss: 1.8023   
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8357 - loss: 0.6510 
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9074 - loss: 0.4071 
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9153 - loss: 0.3236 
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9552 - loss: 0.2471 


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4330 - loss: 1.9458   
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8342 - loss: 0.8479 
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8926 - loss: 0.4383
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9323 - loss: 0.3061
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9475 - loss: 0.2687


Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2444 - loss: 4.2218   
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7113 - loss: 2.2391 
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8036 - loss: 1.7198 
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8630 - loss: 1.4868
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8894 - loss: 1.2979 


###  Оценка моделей

In [13]:
# Шаг 4: Оценка моделей
# Генерация противоречивых примеров для тестирования
adversarial_test_images = create_adversarial_samples(base_model, test_images, test_labels, eps=0.1)

# Оценка базовой модели
print("Точность базовой модели на противоречивых примерах:")
base_model.evaluate(adversarial_test_images, test_labels)

# Оценка модели с обучением на противоречивых примерах
print("Точность модели с обучением на противоречивых примерах:")
adv_model = tf.keras.models.load_model('adversarial_trained_model.h5')
adv_model.evaluate(adversarial_test_images, test_labels)

# Оценка модели с градиентной маскировкой
print("Точность модели с градиентной маскировкой:")
masked_model = tf.keras.models.load_model('masked_model.h5')
masked_model.evaluate(adversarial_test_images, test_labels)

# Оценка модели с регуляризацией и нормализацией
print("Точность модели с регуляризацией и нормализацией:")
regularized_model = tf.keras.models.load_model('regularized_model.h5')
regularized_model.evaluate(adversarial_test_images, test_labels)


Точность базовой модели на противоречивых примерах:
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.1050 - loss: 3.1505


Точность модели с обучением на противоречивых примерах:
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 571us/step - accuracy: 0.5820 - loss: 1.1971


Точность модели с градиентной маскировкой:
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 625us/step - accuracy: 0.3688 - loss: 1.9370
Точность модели с регуляризацией и нормализацией:


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 968us/step - accuracy: 0.5392 - loss: 2.2045


[2.0966432094573975, 0.588699996471405]

 ## Вывод 

В ходе эксперимента была проведена оценка эффективности различных методов защиты моделей от атак с использованием противоречивых примеров. Полученные результаты позволяют сделать следующие выводы:

- 1.Базовая модель продемонстрировала точность 10,5% и уровень потерь 3,15, что подтверждает её высокую уязвимость к FGSM-атакам. Это свидетельствует о недостаточной устойчивости модели без применения дополнительных механизмов защиты.
- 2.Модель с обучением на противоречивых примерах (Adversarial Training) показала заметное улучшение: точность составила 58,2%, а уровень потерь снизился до 1,19. Данный метод продемонстрировал свою эффективность в повышении устойчивости модели к атакам.
- 3.Модель с градиентной маскировкой (Gradient Masking) достигла точности 36,88% и уровня потерь 1,94. Несмотря на некоторый прогресс по сравнению с базовой моделью, её результаты уступают модели с обучением на противоречивых примерах, что указывает на ограниченность метода.
- 4.Модель с регуляризацией и нормализацией показала точность 53,92% и уровень потерь 2,20, что близко к результатам модели с обучением на противоречивых примерах. Это подчёркивает значимость регуляризации и нормализации для повышения устойчивости к атакам.

Таким образом, наиболее эффективным методом защиты оказалась Adversarial Training, что подтверждается самой высокой точностью. В то же время, использование регуляризации и нормализации также демонстрирует потенциал для повышения устойчивости модели. В дальнейшем рекомендуется исследовать комбинирование различных методов защиты, таких как Adversarial Training, Gradient Masking и регуляризация, для создания более устойчивых архитектур, способных сохранять точность даже в условиях атак.